In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 打开显示
tf.debugging.set_log_device_placement(True)

In [3]:
# 显示电脑上的物理GPU
gpus = tf.config.experimental.list_physical_devices('GPU')

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# 需要在GPU没有初始化之前执行. 
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048),
    tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
)

In [6]:
# 逻辑GPU
tf.config.experimental.list_logical_devices('GPU')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
 LogicalDevice(name='/device:GPU:1', device_type='GPU'),
 LogicalDevice(name='/device:GPU:2', device_type='GPU')]

In [7]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [8]:
x_train.astype(np.float32).reshape(-1, 784).shape

(55000, 784)

In [9]:
# 标准化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(55000, -1))
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(5000, -1))

x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(10000, -1))

In [10]:
def make_dataset(data, target, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size).prefetch(50)
    return dataset

batch_size = 64
epochs = 20
train_dataset = make_dataset(x_train_scaled, y_train, epochs, batch_size)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DummySeedGenerator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ShuffleDatasetV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [11]:
strategy = tf.distribute.MirroredStrategy()

# 把模型定义的代码放到strategy.scope中. 
with strategy.scope():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(512, activation='relu', input_shape=(784,)))
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/d

Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in devic

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [13]:
eval_dataset = make_dataset(x_valid_scaled, y_valid, epochs=1, batch_size=32, shuffle=False)

Executing op TensorSliceDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op BatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0


In [14]:
history = model.fit(train_dataset, 
         steps_per_epoch=x_train_scaled.shape[0] // batch_size,
         epochs=10,
         validation_data=eval_dataset)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ComputeBatchSize in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op NotEqual in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RebatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AutoShardDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in devi

Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp

Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
 10/859 [..............................] - ETA: 15s - loss: 1.5890 - accuracy: 0.4511Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
 14/859 [..............................] - ETA: 15s - loss: 1.4561 - accuracy: 0.4964Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train

Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
 78/859 [=>............................] - ETA: 10s - loss: 0.9288 - accuracy: 0.6712Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
 83/859 [=>............................] - ETA: 10s - loss: 0.9147 - accuracy: 0.6760Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train

...

732/859 [========================>.....] - ETA: 1s - loss: 0.1584 - accuracy: 0.9401Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
736/859 [========================>.....] - ETA: 1s - loss: 0.1584 - accuracy: 0.9401Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
740/859 [=======================

Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
802/859 [===========================>..] - ETA: 0s - loss: 0.1582 - accuracy: 0.9401Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
807/859 [===========================>..] - ETA: 0s - loss: 0.1582 - accuracy: 0.9401Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_function_2375 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_train_f

Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op MultiDeviceIteratorToStringHandle in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GeneratorDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op GeneratorDataset in device /job:localho

Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

In [15]:
history.history

{'loss': [0.43916183710098267,
  0.3232947587966919,
  0.2847830653190613,
  0.2589053511619568,
  0.2374861240386963,
  0.2160569578409195,
  0.20080789923667908,
  0.1823727935552597,
  0.17196638882160187,
  0.15663747489452362],
 'accuracy': [0.840912401676178,
  0.8802386522293091,
  0.8932806849479675,
  0.903376042842865,
  0.9109975099563599,
  0.9179642200469971,
  0.9240031838417053,
  0.9312973022460938,
  0.9350807666778564,
  0.940373957157135],
 'val_loss': [0.36020052433013916,
  0.32161685824394226,
  0.32707804441452026,
  0.3387181758880615,
  0.3428051471710205,
  0.3313935101032257,
  0.33591699600219727,
  0.36321017146110535,
  0.38238441944122314,
  0.40906164050102234],
 'val_accuracy': [0.8690000176429749,
  0.8812000155448914,
  0.8831999897956848,
  0.8853999972343445,
  0.8781999945640564,
  0.8884000182151794,
  0.8898000121116638,
  0.8884000182151794,
  0.8871999979019165,
  0.8880000114440918]}

In [16]:
model.evaluate(eval_dataset)

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Equal in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op GreaterEqual in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ComputeBatchSize in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op NotEqual in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RebatchDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AutoShardDa

Executing op AddN in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:1
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:2
Executing op AddN in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost

 63/157 [===========>..................] - ETA: 0s - loss: 0.4155 - accuracy: 0.8869Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
 69/157 [============>.................] - ETA: 0s - loss: 0.4088 - accuracy: 0.8872Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4

136/157 [========================>.....] - ETA: 0s - loss: 0.4114 - accuracy: 0.8869Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
142/157 [==========================>...] - ETA: 0s - loss: 0.4124 - accuracy: 0.8864Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4905 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op __inference_test_function_4

[0.40906164050102234, 0.8880000114440918]